# Entrenamiento de los agentes basados en redes neuronales

Para entrenar a los agentes, deberemos definir un set de datos con estados del juego y un set de etiquetas con la acción preferida (representada en un vector de largo 5 con ceros en todas las posiciones excepto la de la acción elegida).

Luego, entrenaremos la red neuronal en este set de datos y la guardaremos en un archivo (``NN[Agent].h5``), el cual podrá ser accedido por el agente en el archivo ``neural.py``.

In [ ]:
# Librerías utilizadas para trabajar en el notebook
import tensorflow as tf # Instalar mediante pip install tensorflow
import numpy as np
import random
import os

In [ ]:
# Funciones auxiliares (que pueden resultar útiles) y rival de baseline, entregado con la tarea
from agents.baseline import BaseCat, BaseMouse
from utils import bfs_search, get_valid_moves

In [ ]:
# Cargamos el mapa de juego, para poder buscar espacios libres dentro de él
lab_map = np.load(os.path.join("game_map.npy"))

In [ ]:
# Obtenemos los espacios libres dentro del mapa
free_positions = []

for x in range(lab_map.shape[0]):
    for y in range(lab_map.shape[1]):
        if lab_map[x, y] == 0:
            free_positions.append((x, y))

free_positions[:5]

## Entrenamiento del gato 🐈

### Creación del set de datos

Generaremos un set de datos ``X`` que contendrá numpy arrays con la posición del gato y el ratón ``(cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1])``.

Además, generaremos un set de etiquetas ``y`` que contendrá numpy arrays con el movimiento a realizar (ej. ``(0, 0, 1, 0, 0)`` significa hacer el movimiento 2).

La forma en que calculas qué acción llevar a cabo dado el estado de juego es tu decisión, puede ser una búsqueda utilizando el algoritmo de BFS entregado, una política manual o una serie de reglas if-else. Lo importante es que este será el comportamiento que la red neuronal del agente tratará de aproximar. Recuerda que dispones del mapa de juego en ``lab_map`` y de un agente contra el que entrenar bajo la clase `BaseMouse`.

In [ ]:
# Instanciamos al jugador oponente
rival = BaseMouse((0, 0))

# Trabajaremos a X e y como listas, en la siguiente celda las transformaremos a un array de numpy
# (los componentes que colocas en ellas si deben ser numpy arrays)
X = []
y = []

# Para cada combinación de posiciones de gato y ratón
for cat_pos in free_positions:
    for mouse_pos in free_positions:
        X.append(np.array([cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1]]))

        # ===== COMPLETAR =====
        # Se debe calcular un vector de accion, correspondiente a la decisión a tomar para añadirlo a la lista y
        label = np.zeros((5))   # Actualmente se trata de un vector de solo ceros, se debe reemplazar la posición del movimiento elegido por un 1
        # =====================

        y.append(label)

In [ ]:
# Transformamos las listas a numpy arrays
X = np.array(X)
y = np.array(y)

### Diseño y entrenamiento de la Red Neuronal

Para entrenar nuestra red neuronal, utilizaremos la librería de [`TensorFlow`](https://www.tensorflow.org/api_docs/python/tf/all_symbols), podemos instalar esta librería mediante la línea `pip install tensorflow`.

TensorFlow es una librería desarrollada por Google que nos permite construir, entrenar e implementar modelos de aprendizaje profundo en Python.

Para esto, TensorFlow nos permite armar una red en forma de "capas", [en el siguiente link encontrarás un tutorial más en detalle de cómo crear un modelo de TensorFlow](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5). Deberemos introducir todas las capas de nuestra red dentro de un objeto `tf.keras.Sequential()`, que recibe de parámetro una lista con todos los elementos de nuestra red.

<br>

A continuación se enumeran los tres principales elementos que utilizaremos en nuestro modelo:
- `tf.keras.Input(shape = a)`: La capa de entrada de la red, es la primera que recibe. Necesita de un parámetro `shape` que determinará la dimensionalidad del vector de entrada, esto será entregado en una tupla `a` (ej. `(5, 2)`).

- `tf.keras.layers.Dense(units = b, activation = 'relu')`: Corresponde a una capa intermedia de la red del tipo *Fully Connected*, le entregaremos un parámetro `units` que determinará cuántas unidades ocultas (neuronas) tendrá la capa (y por tanto, el número de componentes en el vector de salida de la capa, en este caso `b`). Además, recibirá una función de activación para cada neurona, esta puede ser del tipo `relu`, `tanh`, `softmax` u otras, para más detalle sobre ellas visitar [el siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/activations). Recomendamos usar `relu` para capas intermedias.

- `tf.keras.layers.Dense(units = c, activation = 'softmax')`: Similar a lo explicado anteriormente, corresponde a una capa densa de activación `softmax`. Esta corresponderá a la capa final del modelo, con `c` el número de elementos que deseamos que nuestro vector de salida tenga. El uso de una función softmax se debe a que esta nos permite llevar la salida de nuestra neurona a un conjunto de probabilidades normalizadas, desde el cual podemos calcular qué acción jugar. [En el siguiente link](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer) se explica el funcionamiento de esta función en mayor detalle.

<br>

Un ejemplo de una red simple sería:
```
model = tf.keras.Sequential([
    tf.keras.Input(shape = a),
    tf.keras.layers.Dense(b, activation = "relu"),
    tf.keras.layers.Dense(c, activation = "softmax")
])
```

In [ ]:
# Recuerda que para esta tarea la entrada es un vector de 4 elementos y buscamos una salida de 5 elementos (la probabilidad de jugar acción posible)

# ===== COMPLETAR =====
# Se debe crear un modelo para nuestra red neuronal, añade cuantas capas ocultas desees, con cuantas neuronas desees
model = tf.keras.Sequential([])
# =====================

TensorFlow nos permite además compilar un modelo, especificando qué optimizador utilizar para su ajuste de parámetros y qué función de pérdida utilizar para hacer la backpropagation.

Compilaremos un modelo de la forma:
``model.compile(optimizer = 'optimizer_name', loss = 'loss_name', metrics = ['accuracy'])``

- Con `optimizer_name` el nombre de algún optimizador (por ejemplo, `sgd` o `adam`, más optimizadores en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers))
- Con `loss_name` el nombre de alguna función de pérdida (por ejemplo, `binary_crossentropy`, más funciones en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/losses))
- Usaremos de métrica el accuracy del modelo, aunque existe [múltiples otras métricas que incluir](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [ ]:
# ===== COMPLETAR =====
# Compila tu modelo de TensorFlow
model
# =====================


Ahora que especificamos cómo vamos calcular el error del modelo y cómo lo usaremos para ajustar sus pesos, es hora de entrenar nuestra red. Para ello, utilizaremos su método `.fit()` de la siguiente forma:

`model.fit(X, y, epochs = a, batch_size = b)`

- `X` corresponde a nuestra matriz de características
- `y` corresponde a nuestro vector de etiquetas para cada muestra
- `epochs` determina cuántas veces iteraremos sobre todo el set de datos
- `batch_size` determina cuántas muestras serán consideradas por propagación del gradiente (un valor mayor significa entrenamiento más rápido pero mayor uso de memoria)

In [ ]:
# ===== COMPLETAR =====
# Entrena tu modelo de TensorFlow utilizando su método .fit()
model
# =====================

Por último, guardaremos el modelo entrenado en el directorio `agents/data/NNCat.h5` para poder acceder a este modelo desde nuestra clase `NNCat`:

In [ ]:
model.save(os.path.join('agents', 'data', 'NNCat.h5'))

## Entrenamiento del ratón 🐁

### Creación del set de datos

Generaremos un set de datos ``X`` que contendrá numpy arrays con la posición del gato y el ratón ``(cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1])``.

Además, generaremos un set de etiquetas ``y`` que contendrá numpy arrays con el movimiento a realizar (ej. ``(1, 0, 0, 0, 0)`` significa hacer el movimiento 0).

La forma en que calculas qué acción llevar a cabo dado el estado de juego es tu decisión. Lo importante es que este será el comportamiento que la red neuronal del agente tratará de aproximar. Recuerda que dispones del mapa de juego en ``lab_map`` y de un agente contra el que entrenar bajo la clase `BaseCat`.

In [ ]:
# Instanciamos al jugador oponente
rival = BaseCat((0, 0))

X = []
y = []

for cat_pos in free_positions:
    for mouse_pos in free_positions:
        X.append(np.array([cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1]]))

        # ===== COMPLETAR =====
        # Se debe calcular un vector de accion, correspondiente a la decisión a tomar para añadirlo a la lista y
        label = np.zeros((5))   # Actualmente se trata de un vector de solo ceros, se debe reemplazar la posición del movimiento elegido por un 1
        # =====================

        y.append(label)

In [ ]:
# Transformamos las listas a numpy arrays
X = np.array(X)
y = np.array(y)

### Diseño y entrenamiento de la Red Neuronal

Para entrenar nuestra red neuronal, utilizaremos la librería de [`TensorFlow`](https://www.tensorflow.org/api_docs/python/tf/all_symbols), podemos instalar esta librería mediante la línea `pip install tensorflow`.

TensorFlow es una librería desarrollada por Google que nos permite construir, entrenar e implementar modelos de aprendizaje profundo en Python.

Para esto, TensorFlow nos permite armar una red en forma de "capas", [en el siguiente link encontrarás un tutorial más en detalle de cómo crear un modelo de TensorFlow](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5). Deberemos introducir todas las capas de nuestra red dentro de un objeto `tf.keras.Sequential()`, que recibe de parámetro una lista con todos los elementos de nuestra red.

<br>

A continuación se enumeran los tres principales elementos que utilizaremos en nuestro modelo:
- `tf.keras.Input(shape = a)`: La capa de entrada de la red, es la primera que recibe. Necesita de un parámetro `shape` que determinará la dimensionalidad del vector de entrada, esto será entregado en una tupla `a` (ej. `(5, 2)`).

- `tf.keras.layers.Dense(units = b, activation = 'relu')`: Corresponde a una capa intermedia de la red del tipo *Fully Connected*, le entregaremos un parámetro `units` que determinará cuántas unidades ocultas (neuronas) tendrá la capa (y por tanto, el número de componentes en el vector de salida de la capa, en este caso `b`). Además, recibirá una función de activación para cada neurona, esta puede ser del tipo `relu`, `tanh`, `softmax` u otras, para más detalle sobre ellas visitar [el siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/activations). Recomendamos usar `relu` para capas intermedias.

- `tf.keras.layers.Dense(units = c, activation = 'softmax')`: Similar a lo explicado anteriormente, corresponde a una capa densa de activación `softmax`. Esta corresponderá a la capa final del modelo, con `c` el número de elementos que deseamos que nuestro vector de salida tenga. El uso de una función softmax se debe a que esta nos permite llevar la salida de nuestra neurona a un conjunto de probabilidades normalizadas, desde el cual podemos calcular qué acción jugar. [En el siguiente link](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer) se explica el funcionamiento de esta función en mayor detalle.

<br>

Un ejemplo de una red simple sería:
```
model = tf.keras.Sequential([
    tf.keras.Input(shape = a),
    tf.keras.layers.Dense(b, activation = "relu"),
    tf.keras.layers.Dense(c, activation = "softmax")
])
```

In [ ]:
# Recuerda que para esta tarea la entrada es un vector de 4 elementos y buscamos una salida de 5 elementos (la probabilidad de jugar acción posible)

# ===== COMPLETAR =====
# Se debe crear un modelo para nuestra red neuronal, añade cuantas capas ocultas desees, con cuantas neuronas desees
model = tf.keras.Sequential([])
# =====================

TensorFlow nos permite además compilar un modelo, especificando qué optimizador utilizar para su ajuste de parámetros y qué función de pérdida utilizar para hacer la backpropagation.

Compilaremos un modelo de la forma:
``model.compile(optimizer = 'optimizer_name', loss = 'loss_name', metrics = ['accuracy'])``

- Con `optimizer_name` el nombre de algún optimizador (por ejemplo, `sgd` o `adam`, más optimizadores en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers))
- Con `loss_name` el nombre de alguna función de pérdida (por ejemplo, `binary_crossentropy`, más funciones en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/losses))
- Usaremos de métrica el accuracy del modelo, aunque existe [múltiples otras métricas que incluir](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [ ]:
# ===== COMPLETAR =====
# Compila tu modelo de TensorFlow
model
# =====================

Ahora que especificamos cómo vamos calcular el error del modelo y cómo lo usaremos para ajustar sus pesos, es hora de entrenar nuestra red. Para ello, utilizaremos su método `.fit()` de la siguiente forma:

`model.fit(X, y, epochs = a, batch_size = b)`

- `X` corresponde a nuestra matriz de características
- `y` corresponde a nuestro vector de etiquetas para cada muestra
- `epochs` determina cuántas veces iteraremos sobre todo el set de datos
- `batch_size` determina cuántas muestras serán consideradas por propagación del gradiente (un valor mayor significa entrenamiento más rápido pero mayor uso de memoria)

In [ ]:
# ===== COMPLETAR =====
# Entrena tu modelo de TensorFlow utilizando su método .fit()
model
# =====================

Por último, guardaremos el modelo entrenado en el directorio `agents/data/NNMouse.h5` para poder acceder a este modelo desde nuestra clase `NNMouse`:

In [ ]:
model.save(os.path.join('agents', 'data', 'NNMouse.h5'))